## EDENAI

In [1]:
import base64
import json
import requests
import os

def base64_to_mp3(audio_string, output_filename):
    """
    Converts a Base64 encoded string representation of an audio file to an MP3 file.

    Args:
    audio_string (str): The Base64 encoded string representation of the audio file.
    output_filename (str): The name of the output MP3 file.
    """
    decoded_audio = base64.b64decode(audio_string)

    with open(output_filename, 'wb') as f:
        f.write(decoded_audio)  
        
def text_to_speech_mp3(text, output_filename, language='en-US', option='MALE'):
    """
    Converts a text string to speech using the Eden AI API and writes the resulting audio to an MP3 file.

    Args:
    text (str): The text string to convert to speech.
    output_filename (str): The name of the output MP3 file.
    api_key (str): Your Eden AI API key.
    language (str): The language code for the speech synthesis. Defaults to 'en-US'.
    option (str): The voice option for the speech synthesis. Defaults to 'MALE'.
    """
    api_key = os.environ.get('EDENAI_API_KEY')
    headers = {"Authorization": f"Bearer {api_key}"}
    url = "https://api.edenai.run/v2/audio/text_to_speech"
    payload = {
        "providers": "google",
        "language": language,
        "option": option,
        "text": text,
    }

    response = requests.post(url, json=payload, headers=headers)
    result = json.loads(response.text)
    # print(result)
    audio_string = result['google']['audio']
    print('cost : ',result['google']['cost'])
    decoded_audio = base64.b64decode(audio_string)

    with open(output_filename, 'wb') as f:
        f.write(decoded_audio)

In [2]:
text = "“Houdini” is the lead single of Eminem’s twelfth studio album The Death of Slim Shady (Coup de Grâce). The track is named after Harry Houdini, a popular magician known for his death-defying stunts like the Chinese Water Torture Cell, which Eminem replicates in the intro to his 1999 track, “Role Model.”"
text_to_speech_mp3(text, "output_tts.mp3")

cost :  0.004848


## times of words

In [3]:
import json
import requests

headers = {"Authorization": f"Bearer {os.environ.get('EDENAI_API_KEY')}"}

url = "https://api.edenai.run/v2/audio/speech_to_text_async"
data = {
    "providers": "OpenAI",
    "language": "en-US",
}

files = {'file': open("audio1.mp3", 'rb')}

response = requests.post(url, data=data, files=files, headers=headers)
result = json.loads(response.text)
print(result)


{'error': {'type': 'Invalid request', 'message': {'providers': ["No such provider 'OpenAI' for 'audio.speech_to_text_async'"]}}}


# OPEN AI

## TTS

In [4]:
from openai import OpenAI

api_key = os.environ.get('OPENAI_API_KEY_LORRAIN')
client = OpenAI(api_key=api_key)

def text_to_speech_mp3(text, output_filename, model="tts-1", voice="alloy"):
    """
    Converts a text string to speech using the OpenAI API and writes the resulting audio to an MP3 file.

    Args:
    text (str): The text string to convert to speech.
    output_filename (str): The name of the output MP3 file.
    model (str): The model to use for speech synthesis. Defaults to 'tts-1'.
    voice (str): The voice to use for speech synthesis. Defaults to 'alloy'.
    """
    global client
    client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY_LORRAIN')) if client is None else client
    speech_file_path = output_filename


    with client.audio.speech.with_streaming_response.create(
        model=model,
        voice=voice,
        input=text,
    ) as response:
        response.stream_to_file(speech_file_path+('.mp3' if not speech_file_path.endswith('.mp3') else ''))


In [5]:
original_text = "Today is a wonderful day to build something people love! Today is a beautiful day"
text_to_speech_mp3(original_text, "output_tts.mp3")

## Times of the text 

In [6]:
from openai import OpenAI
import os

def getTextTimingsOfMp3(mp3file):
    global client
    client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY_LORRAIN')) if client is None else client
    
    audio_file = open(mp3file, "rb")
    transcript = client.audio.transcriptions.create(
        file=audio_file,
        model="whisper-1",
        response_format="verbose_json",
        timestamp_granularities=["word"]
    )

    return transcript



In [7]:
text = getTextTimingsOfMp3("output_tts.mp3")
print(text)

Transcription(text='Today is a wonderful day to build something people love. Today is a beautiful day.', task='transcribe', language='english', duration=4.820000171661377, words=[{'word': 'Today', 'start': 0.0, 'end': 0.3400000035762787}, {'word': 'is', 'start': 0.3400000035762787, 'end': 0.5600000023841858}, {'word': 'a', 'start': 0.5600000023841858, 'end': 0.6800000071525574}, {'word': 'wonderful', 'start': 0.6800000071525574, 'end': 0.9800000190734863}, {'word': 'day', 'start': 0.9800000190734863, 'end': 1.2599999904632568}, {'word': 'to', 'start': 1.2599999904632568, 'end': 1.440000057220459}, {'word': 'build', 'start': 1.440000057220459, 'end': 1.6200000047683716}, {'word': 'something', 'start': 1.6200000047683716, 'end': 2.059999942779541}, {'word': 'people', 'start': 2.059999942779541, 'end': 2.4000000953674316}, {'word': 'love', 'start': 2.4000000953674316, 'end': 2.759999990463257}, {'word': 'Today', 'start': 3.4200000762939453, 'end': 3.5999999046325684}, {'word': 'is', 'star

In [8]:
def verify_transcription(original_text, transcription):
    """
    Verifies that the text is the same after the tts and stt.

    Args:
    original_text (str): The original text.
    transcription (dict): The transcription dictionary.

    Returns:
    int: ratio of the original text and the transcription text.
    """
    print(transcription.text)
    print(original_text)
    transcription_text = transcription.text.lower().replace('!', '').replace('.', '').replace(',', '').replace('?', '')
    original_text = original_text.lower().replace('!', '').replace('.', '').replace(',', '').replace('?', '')
    
    if transcription_text == original_text:
        return 1
    
    # correct_words = 0
    for word in original_text.split():
        transcription_text = transcription_text.replace(word, '', 1)
#
    return 1 - len(transcription_text) / len(original_text)

print(verify_transcription(original_text, text))

Today is a wonderful day to build something people love. Today is a beautiful day.
Today is a wonderful day to build something people love! Today is a beautiful day
1


In [9]:
def get_sentence_timings(transcription, sentence):
    """
    Returns the start and end timings of a sentence in a transcription object.

    Args:
    transcription (Transcription): The transcription object.
    sentence (str): The sentence to search for.

    Returns:
    tuple: A tuple containing the start and end timings of the sentence.
    """
    words = transcription.words
    sentence_words = sentence.split()
    sentence_start = None
    sentence_end = None

    i = 0
    while i < len(words):
        if words[i]['word'].lower() == sentence_words[0].lower():
            j = 1
            while j < len(sentence_words) and i + j < len(words) and words[i + j]['word'].lower() == sentence_words[j].lower():
                j += 1
            if j == len(sentence_words):
                sentence_start = words[i]['start']
                sentence_end = words[i + j - 1]['end']
                break
        i += 1

    return sentence_start, sentence_end

In [13]:
print(text)
sentence_start, sentence_end = get_sentence_timings(text, "Today is a beautiful day")
print(f"Sentence start: {sentence_start}, sentence end: {sentence_end}")

Transcription(text='Today is a wonderful day to build something people love. Today is a beautiful day.', task='transcribe', language='english', duration=4.820000171661377, words=[{'word': 'Today', 'start': 0.0, 'end': 0.3400000035762787}, {'word': 'is', 'start': 0.3400000035762787, 'end': 0.5600000023841858}, {'word': 'a', 'start': 0.5600000023841858, 'end': 0.6800000071525574}, {'word': 'wonderful', 'start': 0.6800000071525574, 'end': 0.9800000190734863}, {'word': 'day', 'start': 0.9800000190734863, 'end': 1.2599999904632568}, {'word': 'to', 'start': 1.2599999904632568, 'end': 1.440000057220459}, {'word': 'build', 'start': 1.440000057220459, 'end': 1.6200000047683716}, {'word': 'something', 'start': 1.6200000047683716, 'end': 2.059999942779541}, {'word': 'people', 'start': 2.059999942779541, 'end': 2.4000000953674316}, {'word': 'love', 'start': 2.4000000953674316, 'end': 2.759999990463257}, {'word': 'Today', 'start': 3.4200000762939453, 'end': 3.5999999046325684}, {'word': 'is', 'star